##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 추정기

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ko/guide/estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ko/guide/estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />GitHub 소스 보기</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/ko/guide/estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />노트북 다운로드하기</a>
  </td>
</table>

Note: 이 문서는 텐서플로 커뮤니티에서 번역했습니다. 커뮤니티 번역 활동의 특성상 정확한 번역과 최신 내용을 반영하기 위해 노력함에도 불구하고 [공식 영문 문서](https://www.tensorflow.org/?hl=en)의 내용과 일치하지 않을 수 있습니다. 이 번역에 개선할 부분이 있다면 [tensorflow/docs](https://github.com/tensorflow/docs) 깃헙 저장소로 풀 리퀘스트를 보내주시기 바랍니다. 문서 번역이나 리뷰에 참여하려면 [docs-ko@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ko)로 메일을 보내주시기 바랍니다.

이 문서에서는 고수준 텐서플로 API인 `tf.estimator` 를 소개합니다. 추정기는 다음 작업들을 캡슐화 합니다.
* 훈련
* 평가
* 예측
* 내보내기

사전 제작된 추정기나 직접 만든 추정기를 사용할 수 있습니다. 모든 추정기들은 `tf.estimator.Estimator` 를 기반으로 합니다.

간단한 예시를 위해 [추정기 튜토리얼](../../tutorials/estimator/linear.ipynb) 을 참조하세요. API 디자인에 대한 개요는 [백서](https://arxiv.org/abs/1708.02637)를 참조하세요.

## 장점

`tf.keras.Model`과 유사하게, `estimator` 는 모델 수준의 추상적 개념입니다. `tf.estimator`는 현재 `tf.keras`에서 아직까지 개발중인 일부 기능들을 제공합니다.

그 기능들은 다음과 같습니다 :

  * 파라미터 서버 기반 훈련
  * 완전한 [TFX](http://tensorflow.org/tfx) 통합.

## 추정기의 기능
추정기는 다음과 같은 이점을 제공합니다 :

* 모델을 변경하지 않고 로컬호스트 또는 분산된 다중 서버 환경에서 추정기 기반 모델을 실행할 수 있습니다. 또한, 모델을 재코딩하지 않고도 CPU, GPU, TPU 에서 추정기 기반 모델을 실행할 수 있습니다.

* 안전한 분산 훈련 루프를 제공하여 아래의 행동들을 언제, 어떻게 해야 하는지를 제어합니다.
    * 데이터 로드
    * 예외 처리
    * 체크포인트 파일 생성 및 장애 복구
    * 텐서보드를 위한 서머리(summary) 저장

추정기를 사용하여 프로그램을 작성할 때, 반드시 모델에서 데이터 입력 파이프라인을 분리해야 합니다. 이 분리는 다른 데이터 세트로 실험할 때의 과정을 단순화합니다.

## 사전 제작된 추정기

사전 제작된 추정기는 텐서플로 API 기반의 작업보다 훨씬 높은 추상화 수준에서 작업할 수 있습니다.
추정기가 모든 파이프라인을 처리하기 때문에 계산 그래프나 세션을 만드는 것에 대해서 더 이상 걱정할 필요가 없습니다. 또한, 사전 제작된 추정기는 최소한의 코드 변경만으로 다른 모델 아키텍처를 실험할 수 있게 해줍니다. 예를 들어 `tf.estimator.DNNClassifier` 는 완전 연결 신경망을 기반으로 한 분류 모델을 훈련시키는 사전 제작된 추정기입니다.

### 사전 제작된 추정기 프로그램의 구조

사전 제작된 추정기에 의존하는 텐서플로 프로그램은 일반적으로 4단계로 구성됩니다.

#### 1. 하나 이상의 데이터세트 임포트 함수를 작성합니다.

예를 들어, 훈련 세트를 가져오는 함수와 테스트 세트를 가져오는 다른 함수를 생성했을 때. 각각의 데이터세트 임포트 함수는 다음 두개의 오브젝트를 반환해야 합니다 :

* 키가 특성 이름이고 값이 해당 특성 데이터를 포함하는 텐서(또는 희소텐서)인 딕셔너리
* 한개 이상의 레이블을 포함하고 있는 텐서

예를 들어, 다음 코드는 입력 함수의 기본 골격을 보여줍니다.

```
def input_fn(dataset):
    ...  # 데이터 세트를 조작하여 특성 딕셔너리와 레이블을 추출합니다.
    return feature_dict, label
```

자세한 내용은 [데이터 가이드](../../guide/data.md)를 참조하세요.

#### 2. 특성 열을 정의합니다.

각 `tf.feature_column`은 특성 이름, 유형 및 입력 전처리를 지정할 수 있습니다. 예를 들어, 다음 예제는 정수 혹은 부동 소수점 데이터를 가지는 3개의 특성 열을 생성합니다. 처음의 두 특성 열은 단순히 특성의 이름과 유형만 지정합니다. 세번째 특성 열은 프로그램이 원본 데이터의 스케일을 조정하기 위해 호출할 람다(lambda) 함수도 지정합니다 :


```
# 3개의 특성 열을 정의합니다. 
population = tf.feature_column.numeric_column('population')
crime_rate = tf.feature_column.numeric_column('crime_rate')
median_education = tf.feature_column.numeric_column(
  'median_education',
  normalizer_fn=lambda x: x - global_education_mean)
```

자세한 내용은 [특성 열 튜토리얼](https://www.tensorflow.org/tutorials/keras/feature_columns)을 참조하세요.

#### 3. 사전 제작된 추정기를 인스턴스화합니다.

예를 들어, 다음은 `LinearClassifier` 라는 이름의 사전 제작된 추정기의 인스턴스로 만드는 샘플입니다 :

```
# 특성 열을 지나면서, 추정기를 인스턴스화합니다.
estimator = tf.estimator.LinearClassifier(
  feature_columns=[population, crime_rate, median_education])
```
자세한 내용은 [선형 분류기 튜토리얼](https://www.tensorflow.org/tutorials/estimator/linear)을 참조하세요.

#### 4. 훈련, 평가, 혹은 추론 메소드를 호출합니다.

예를 들어, 모든 추정기는 모델을 훈련시키는 `train` 메소드를 제공합니다.

```
# `input_fn` 은 1단계에서 만들어진 함수입니다.
estimator.train(input_fn=my_training_set, steps=2000)
```
아래에서는 이에 대한 예를 볼 수 있습니다.

### 사전 제작된 추정기의 이점

사전 제작된 추정기는 모범 사례를 담고 있으며, 다음과 같은 이점들을 제공합니다 :

* 단일 시스템 또는 클러스터에서 전략을 구현하여 계산 그래프에서 실행할 위치를 결정하는 모범 사례.
* 이벤트(서머리) 작성과 보편적으로 유용한 서머리에 대한 모범 사례.

사전 작성된 추정기를 사용하지 않을 경우, 이전 기능들을 직접 구현해야 합니다.

## 사용자 정의 추정기

사전 제작된 추정기나 사용자 정의 추정기든지 상관없이 핵심은 훈련, 평가 및 예측을 위한 그래프를 만드는 *모델 함수* 입니다. 사전 제작된 추정기를 사용하는 경우, 다른 누군가가 이미 모델 함수를 구현해 놓았습니다. 사용자 지정 추정기를 사용할 때에는 모델 함수를 직접 작성해야 합니다.

## 추천하는 작업 방법

1. 적합한 사전 제작된 추정기가 존재한다고 가정할 때, 첫 번째 모델을 제작하고 그 결과를 사용하여 기준선을 설정하세요.
2. 이 사전 제작된 추정기를 사용하여 데이터의 무결성 및 신뢰성을 비롯한 전체 파이프라인을 구축하고 테스트하세요.
3. 적절한 사전 제작된 추정기가 사용자 정의 추정기를 대체할 수 있는 경우, 사전 제작된 추정기가 어떠한 최상의 결과를 산출하는지 실험을 진행해 보세요.
4. 가능한 사용자 정의 추정기를 구축하여 모델을 더욱 개선해보세요.

In [12]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [13]:
try:
  # %tensorflow_version은 오직 코랩에만 존재합니다.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [14]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

## Keras 모델에서 추정기 만들기

기존 Keras 모델을 `tf.keras.estimator.model_to_estimator`를 사용하여 추정기로 변환할 수 있습니다. 이 방식을 사용하면 Keras 모델이 '분산 훈련'과 같은 추정기의 장점을 가질 수 있게 됩니다.

Keras MobileNet V2 모델을 인스턴스화하고, 옵티마이저, 손실 및 측정 지표 등을 사용하여 모델을 컴파일 하고 훈련시키세요 :

In [15]:
keras_mobilenet_v2 = tf.keras.applications.MobileNetV2(
    input_shape=(160, 160, 3), include_top=False)

estimator_model = tf.keras.Sequential([
    keras_mobilenet_v2,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='softmax')
])

# 모델 컴파일
estimator_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

컴파일된 Keras 모델에서 `estimator`를 생성합니다. Keras 모델의 초기 모델 상태는 생성된 `estimator`에 보존됩니다 :

In [16]:
est_mobilenet_v2 = tf.keras.estimator.model_to_estimator(keras_model=estimator_model)

파생된 `Estimator`를 다른`Estimator`처럼 동일하게 사용할 수 있습니다.

In [8]:
IMG_SIZE = 160  # 모든 이미지는 160x160 크기로 재설정됩니다.

def preprocess(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

In [9]:
def train_input_fn(batch_size):
  data = tfds.load('cats_vs_dogs', as_supervised=True)
  train_data = data['train']
  train_data = train_data.map(preprocess).shuffle(500).batch(batch_size)
  return train_data

훈련을 위해 추정기의 훈련 함수를 호출하세요 :

In [10]:
est_mobilenet_v2.train(input_fn=lambda: train_input_fn(32), steps=500)

마찬가지로, 평가를 위해 추정기의 평가 함수를 호출하세요 :

In [0]:
est_mobilenet_v2.evaluate(input_fn=lambda: train_input_fn(32), steps=10)

자세한 내용은 `tf.keras.estimator.model_to_estimator`의 설명서를 참조하세요.